In [3]:
!where python

C:\Users\Win\Desktop\Project\jupyter_env\jupyter\Scripts\python.exe
C:\Users\Win\AppData\Local\Programs\Python\Python311\python.exe
C:\Users\Win\AppData\Local\Microsoft\WindowsApps\python.exe


In [4]:
!pip install django
!pip install celery
!pip install mysqlclient
!pip install redis



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached celery-5.4.0-py3-none-any.whl (425 kB)
  Using cached billiard-4.2.1-py3-none-any.whl (86 kB)
  Using cached kombu-5.4.2-py3-none-any.whl (201 kB)
  Using cached vine-5.1.0-py3-none-any.whl (9.6 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached click_didyoumean-0.3.1-py3-none-any.whl (3.6 kB)
  Using cached click_repl-0.3.0-py3-none-any.whl (10 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached amqp-5.2.0-py3-none-any.whl (50 kB)



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


                                              0.0/261.3 kB ? eta -:--:--
     --------------------                   143.4/261.3 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 261.3/261.3 kB 3.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install -r requirements.txt

  Using cached cron_descriptor-1.4.5-py3-none-any.whl (50 kB)
  Using cached django_celery_beat-2.7.0-py3-none-any.whl (94 kB)
  Using cached django_redis-5.4.0-py3-none-any.whl (31 kB)
  Using cached django_timezone_field-7.0-py3-none-any.whl (13 kB)
  Using cached djangorestframework-3.15.2-py3-none-any.whl (1.1 MB)
  Using cached python_crontab-3.2.0-py3-none-any.whl (27 kB)
  Using cached redis-5.1.0-py3-none-any.whl (261 kB)
  Attempting uninstall: redis
    Found existing installation: redis 5.1.1
    Uninstalling redis-5.1.1:
      Successfully uninstalled redis-5.1.1



[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import sys
import django
from django.apps import apps

# Set the path to your Django project
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stocks_simplified.settings')

# Only run setup if Django is not already set up
if not apps.ready:
    django.setup()


In [2]:
from stockdata.models import HistoricalData 

import pandas as pd 

from asgiref.sync import sync_to_async 

  

async def fetch_data(): 

    # Fetch data asynchronously 

    data = await sync_to_async(list)(HistoricalData.objects.values( 

        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume' 

    )) 

     

    # Convert to pandas DataFrame 

    df = pd.DataFrame(data) 

     

    # Return or print the first few rows of the DataFrame 

    return df.head() 

  

# Call the asynchronous function 

df_head = await fetch_data() 

print(df_head) 

  stock__symbol        date open_price close_price high_price low_price  \
0          AAPL  2024-09-27     228.46      227.79     229.52    227.30   
1          AAPL  2024-09-26     227.30      227.52     228.50    225.41   
2          AAPL  2024-09-25     224.93      226.37     227.29    224.02   
3          AAPL  2024-09-24     228.65      227.37     229.35    225.73   
4          AAPL  2024-09-23     227.34      226.47     229.45    225.81   

     volume  
0  34025967  
1  36636707  
2  42308715  
3  43556068  
4  54146023  


In [3]:
from stockdata.models import HistoricalData
import pandas as pd
from asgiref.sync import sync_to_async
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Step 1: Fetch data from the database asynchronously
async def fetch_data():
    # Fetch the stock data
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    return df

# Step 2: Prepare data for each stock symbol
def prepare_lstm_data(df, time_step=60):
    stock_symbols = df['stock__symbol'].unique()
    predictions = {}

    for symbol in stock_symbols:
        stock_data = df[df['stock__symbol'] == symbol]
        close_prices = stock_data['close_price'].values.reshape(-1, 1)

        # Normalize data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)

        # Prepare training data
        x_train, y_train = [], []
        for i in range(len(scaled_data) - time_step):
            x_train.append(scaled_data[i:i+time_step, 0])
            y_train.append(scaled_data[i + time_step, 0])
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

        # Step 3: Build and train LSTM model for each stock
        model = Sequential()
        model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1)

        # Step 4: Predict the next day's price for each stock
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, -1, 1)
        predicted_price = model.predict(last_60_days)
        predicted_price_actual = scaler.inverse_transform(predicted_price)

        # Store predictions
        predictions[symbol] = predicted_price_actual[0][0]

        # Step 5: Save the trained model to a file for later use
        model_filename = f'model_{symbol}.pkl'  # Save each model with the stock symbol name
        with open(model_filename, 'wb') as model_file:
            pickle.dump(model, model_file)  # Save the model to a .pkl file

    return predictions

# Step 5: Run the prediction process
df = await fetch_data()  # Fetch the data
predictions = prepare_lstm_data(df)  # Prepare and predict

# Print predictions for all stocks
print(predictions)

C:\Users\Win\Desktop\Project\jupyter_env\jupyter\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.1617
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step


NameError: name 'pickle' is not defined

In [7]:
import os
import sys
import django
from django.apps import apps
import pickle  # Add this import to resolve the NameError
import pandas as pd
from asgiref.sync import sync_to_async
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input  # Add Input import
import numpy as np

# Set the path to your Django project
sys.path.append('C:/Users/Win/Desktop/Project/Stock Project')

# Set the Django settings module
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'stocks_simplified.settings')

# Only run setup if Django is not already set up
if not apps.ready:
    django.setup()

print("Django setup complete")

# Step 1: Fetch data from the database asynchronously
async def fetch_data():
    print("Fetching data from the database...")
    # Fetch the stock data
    data = await sync_to_async(list)(HistoricalData.objects.values(
        'stock__symbol', 'date', 'open_price', 'close_price', 'high_price', 'low_price', 'volume'
    ))

    # Convert to pandas DataFrame
    df = pd.DataFrame(data)
    print(f"Fetched {len(df)} records from the database")
    return df

# Step 2: Prepare data for each stock symbol
def prepare_lstm_data(df, time_step=60):
    print("Preparing LSTM data...")
    stock_symbols = df['stock__symbol'].unique()
    predictions = {}

    for symbol in stock_symbols:
        print(f"Processing stock symbol: {symbol}")
        stock_data = df[df['stock__symbol'] == symbol]
        close_prices = stock_data['close_price'].values.reshape(-1, 1)

        # Normalize data
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(close_prices)

        # Prepare training data
        x_train, y_train = [], []
        for i in range(len(scaled_data) - time_step):
            x_train.append(scaled_data[i:i+time_step, 0])
            y_train.append(scaled_data[i + time_step, 0])
        
        x_train, y_train = np.array(x_train), np.array(y_train)
        x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)

        # Step 3: Build and train LSTM model for each stock
        print(f"Building LSTM model for {symbol}")
        model = Sequential()
        model.add(Input(shape=(x_train.shape[1], 1)))  # Replace input_shape argument with Input layer
        model.add(LSTM(50, return_sequences=True))
        model.add(LSTM(50, return_sequences=False))
        model.add(Dense(25))
        model.add(Dense(1))

        model.compile(optimizer='adam', loss='mean_squared_error')
        model.fit(x_train, y_train, batch_size=1, epochs=1)

        # Step 4: Predict the next day's price for each stock
        print(f"Predicting next price for {symbol}")
        last_60_days = scaled_data[-time_step:]
        last_60_days = last_60_days.reshape(1, -1, 1)
        predicted_price = model.predict(last_60_days)
        predicted_price_actual = scaler.inverse_transform(predicted_price)

        # Store predictions
        predictions[symbol] = predicted_price_actual[0][0]

        # Step 5: Save the trained model to a file for later use
        model_filename = f'model_{symbol}.pkl'  # Save each model with the stock symbol name
        with open(model_filename, 'wb') as model_file:
            pickle.dump(model, model_file)  # Save the model to a .pkl file

        print(f"Model for {symbol} saved to {model_filename}")

    print("LSTM data preparation complete")
    return predictions

# Step 5: Run the prediction process
df = await fetch_data()  # Fetch the data
predictions = prepare_lstm_data(df)  # Prepare and predict

# Print predictions for all stocks
print("Predictions for all stocks:", predictions)


Django setup complete
Fetching data from the database...
Fetched 115 records from the database
Preparing LSTM data...
Processing stock symbol: AAPL
Building LSTM model for AAPL
55/55 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.1015
Predicting next price for AAPL
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step
Model for AAPL saved to model_AAPL.pkl
LSTM data preparation complete
Predictions for all stocks: {'AAPL': 215.03683}
